Import necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

Load and preprocess dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Define layers

In [ ]:
input_layer = Input(shape=(28, 28, 1))
flatten = Flatten()(input_layer)
dense1 = Dense(128, activation='relu')(flatten)
dense2 = Dense(64, activation='relu')(dense1)
intermediate_layer = Dense(32, activation='relu')(dense2)
output_layer = Dense(10, activation='softmax')(intermediate_layer)


Create model

In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []

for epoch in range(5):
    history = model.fit(train_images, train_labels, epochs=1, batch_size=64, validation_data=(test_images, test_labels))

    train_loss_history.append(history.history['loss'][0])
    train_accuracy_history.append(history.history['accuracy'][0])

    val_loss_history.append(history.history['val_loss'][0])
    val_accuracy_history.append(history.history['val_accuracy'][0])

    for layer in model.layers:
        print(layer.name)
        if layer.get_weights():
            weights, biases = layer.get_weights()
            print("Weights shape:", weights.shape)
            print("Biases shape:", biases.shape)
            print("Weights:", weights)
            print("Biases:", biases)
        else:
            print("No parameters")

Show results

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, 6), train_loss_history, label='Train Loss')
plt.plot(range(1, 6), val_loss_history, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, 6), train_accuracy_history, label='Train Accuracy')
plt.plot(range(1, 6), val_accuracy_history, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()